In [ ]:
with open('/home/oda/hack/new_data_set.txt') as f:
	content = f.readlines()
content = [x.strip() for x in content]
content = list(set(content))
content = [x for x in content if x.startswith('[http')]
content

In [1]:
from urllib.parse import urlparse, parse_qs
def custom_tokenize_url(url):
    tokens = []
    parsed_url = urlparse(url)
    
    # Tokenize the domain
    path_tokens = parsed_url.path.split('/')
    path_tokens = path_tokens[1:]
    tokens.extend(filter(None, path_tokens))
    # Tokenize the query string
    query_tokens = []
    query_params = parse_qs(parsed_url.query)
    for param, values in query_params.items():
        query_tokens.append(param)
        query_tokens.extend(values)
    tokens.extend(query_tokens)
    
    # Remove empty tokens and return
    return ' '.join(filter(None, tokens))

tokenized = custom_tokenize_url('https://www.amazon.com/s?k=iphone+11&ref=nb_sb_noss_2')
print(tokenized)

s k iphone 11 ref nb_sb_noss_2


In [ ]:
from urllib.parse import urlparse, parse_qs

def custom_tokenize_url(url):
    tokens = []
    parsed_url = urlparse(url)
    
    # Tokenize the domain
    path_tokens = parsed_url.path.split('/')
    path_tokens = path_tokens[1:]
    tokens.extend(filter(None, path_tokens))
    # Tokenize the query string
    query_tokens = []
    query_params = parse_qs(parsed_url.query)
    for param, values in query_params.items():
        query_tokens.append(param)
        query_tokens.extend(values)
    tokens.extend(query_tokens)
    
    # Remove empty tokens and return
    return ' '.join(filter(None, tokens))

content = [x.strip() for x in content]
import re
import pandas as pd
new_df = pd.DataFrame(columns=['url', 'label', 'name', 'category'])
index = 0

for i in range(len(content)):
    print(i)
    split = content[i].replace('[', '').replace(']', '').split(',')
    if len(split) == 4:
        new_df.loc[i] = [split[0], split[1], split[2], split[3]]


new_df['encoded_label'] = [1 if 'product' in x else 0 for x in new_df['label']]
new_df['url'] = [x.replace('https://', '').replace('"', '').strip() for x in new_df['url']]
new_df['url_tokens'] = new_df['url'].apply(custom_tokenize_url)


In [ ]:
# we now have to build the pipeline
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer, AutoModelForSequenceClassification, AutoTokenizer
from sklearn.model_selection import train_test_split
from datasets import Dataset
import pyarrow as pa
from transformers import TrainingArguments, Trainer
import torch
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

class TrainModel():
	def __init__(self):
		self.model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
		self.tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

	def process_data(self, row):
		text = row['url_tokens']
		text = str(text)
		text = ' '.join(text.split())
		encodings = self.tokenizer(text,truncation=True, padding='max_length', max_length=32)
		encodings['label'] = row['encoded_label']
		encodings['url'] = row['url']
		return encodings

	def prepare_data(self, df):
		processed_data = [self.process_data(df.iloc[i]) for i in range(1,df.shape[0])]
		new_df = pd.DataFrame(processed_data)
		train_df, valid_df = train_test_split(
			new_df,
			test_size=0.19,
			random_state=2022
		)
		train_hg = Dataset(pa.Table.from_pandas(train_df)).shuffle(seed=2022)
		valid_hg = Dataset(pa.Table.from_pandas(valid_df)).shuffle(seed=2022)
		return train_hg, valid_hg

	def compute_metrics(self, pred):
		labels = pred.label_ids
		preds = pred.predictions.argmax(-1)
		precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
		acc = accuracy_score(labels, preds)
		return {
			'accuracy': acc,
			'f1': f1,
			'precision': precision,
			'recall': recall
		}

	def train_model(self, train_hg, valid_hg, output_dir, output_logs):
		device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
		self.model.to(device)
		
		for param in self.model.base_model.parameters():
			param.requires_grad = False

		for param in self.model.classifier.parameters():
			param.requires_grad = True
		
		training_args = TrainingArguments(
			output_dir=output_dir,
			num_train_epochs=20,
			per_device_train_batch_size=16,
			per_device_eval_batch_size=64,
			warmup_steps=1000,
			logging_dir=output_logs ,  # Specify GPU device
		)

		trainer = Trainer(
			model=self.model,
			args=training_args,
			train_dataset=train_hg,
			eval_dataset=valid_hg,
			compute_metrics=self.compute_metrics,
			tokenizer=self.tokenizer
			)
		trainer.train()
		trainer.save_model('150k_model2')
		return trainer

trainer = TrainModel()
train_hg, valid_hg = trainer.prepare_data(new_df)

In [ ]:
train_model = trainer.train_model(train_hg, valid_hg, 'output_dir3', './output_log3')

In [ ]:
def get_prediction(tokenizer, model, text):
	encoding = tokenizer(text, return_tensors='pt', truncation=True, padding='max_length', max_length=32)
	outputs = model(**encoding)
	logits = outputs.logits
	probs = torch.nn.functional.softmax(logits, dim=-1)
	label = torch.argmax(probs).item()
	return label

In [ ]:
custom_tokenize_url('https://www.amazon.com/s?k=iphone+11&ref=nb_sb_noss_2')

In [ ]:
valid_hg

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
model = AutoModelForSequenceClassification.from_pretrained('/home/oda/hack/output_dir2/checkpoint-150500')

get_prediction(tokenizer, model, custom_tokenize_url('https://ocheche.store/sun/ru/tproduct/507267910-936288484151-eyerepublic'))

In [ ]:
count = 0
for i in range(1,valid_hg.shape[0]):
	print(i)
	prediction = get_prediction(tokenizer, model, custom_tokenize_url(valid_hg[i]['url']))
	if prediction == valid_hg[i]['label']:
		count += 1
	
count

In [ ]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score
y_test = [get_prediction(tokenizer, model, custom_tokenize_url(valid_hg[i]['url'])) for i in range(valid_hg.shape[0])]
y_pred = valid_hg['label']

accuracy = accuracy_score(y_pred, y_test)
precision = precision_score(y_pred, y_test)
recall = recall_score(y_pred, y_test, average='macro')
f1 = f1_score(y_pred, y_test, average='macro')

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1: {f1}')


In [ ]:
range(valid_hg.shape[0])